In [28]:
import torchtext; torchtext.disable_torchtext_deprecation_warning()

In [29]:
# Sentiment Analysis

from torchtext.datasets import IMDB

train_dataset = IMDB(split='train')
test_dataset = IMDB(split='test')

In [31]:
# Preprocessing

# Create the datasets
import torch
from torch.utils.data.dataset import random_split
torch.manual_seed(1)
train_dataset, valid_dataset = random_split(
    # cut to make the training faster
    list(train_dataset)[:2500], [2000, 500]
)

In [32]:
# Find unique tokens (words)

import re
from collections import Counter, OrderedDict


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall(
        '(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower()
    )
    text = re.sub('[\W]+', ' ', text.lower()) + \
           ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

token_counts = Counter()
for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)

print('Vocab-size:', len(token_counts))

Vocab-size: 23447


In [33]:
# Encoding each unique token into integers

import torch.nn as nn
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(
    token_counts.items(),
    key=lambda x: x[1], reverse=True
)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = vocab(ordered_dict)
vocab.insert_token('<pad>', 0)
vocab.insert_token('<unk>', 1)
vocab.set_default_index(1)
print([vocab[token] for token in [
    'this', 'is','an', 'example'
]])

text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1. if x == 'pos' else 0.

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(
            text_pipeline(_text),
            dtype=torch.int64
        )
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list,
        batch_first=True
    )
    return padded_text_list, label_list, lengths

from torch.utils.data import DataLoader
dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_batch
)

[11, 7, 37, 402]


In [34]:
# Padding (making the len the same)

text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[   11,    21,     7,  ...,     0,     0,     0],
        [  756,     9,    73,  ...,     0,     0,     0],
        [ 4521,  6588,  1437,  ...,     0,     0,     0],
        [13363,  9801,   653,  ...,  9813,  9814, 13380]])
tensor([0., 0., 0., 0.])
tensor([221, 132, 335, 440])
torch.Size([4, 440])


In [35]:
batch_size = 32
train_dl = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_batch
)
valid_dl = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_batch
)
test_dl = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_batch
)

In [36]:
# Feature embedding
embedding = nn.Embedding(
    num_embeddings=10,
    embedding_dim=3,
    padding_idx=0
)

text_encoded_input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 0]])
print(embedding(text_encoded_input))

tensor([[[ 0.2835,  0.4542,  0.2693],
         [ 1.1386, -0.1532,  0.5672],
         [-0.6690,  0.5642,  0.0668],
         [-0.0070, -0.4395, -1.8729]],

        [[-0.6690,  0.5642,  0.0668],
         [-0.5165, -0.9710,  0.2513],
         [ 1.1386, -0.1532,  0.5672],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)


In [37]:
# Base model

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        # self.rnn = nn.GRU(
        # self.rnn = nn.LSTM(
        self.rnn = nn.RNN(
            input_size,
            hidden_size,
            num_layers=2,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        _, hidden = self.rnn(x)
        # We use the final hidden state from the last hidden layer
        #   as the input to the fully connected layer
        out = hidden[-1, :, :]
        out = self.fc(out)
        return out

model = RNN(64, 32)
print(model)
model(torch.randn(5, 3, 64))

RNN(
  (rnn): RNN(64, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


tensor([[ 0.0944],
        [ 0.5408],
        [ 0.2953],
        [ 0.0781],
        [-0.1059]], grad_fn=<AddmmBackward0>)

In [38]:
# Sentiment analysis model

class RNN(nn.Module):
    def __init__(
        self, 
        vocab_size, 
        embed_dim, 
        rnn_hidden_size, 
        fc_hidden_size
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            padding_idx=0
        )
        self.rnn = nn.LSTM(
            embed_dim, 
            rnn_hidden_size,
            batch_first=True,
            bidirectional=True
        )
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(
            out,
            lengths.cpu().numpy(),
            enforce_sorted=False,
            batch_first=True
        )
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out


vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64
torch.manual_seed(1)
model = RNN(
    vocab_size,
    embed_dim,
    rnn_hidden_size,
    fc_hidden_size,
)
model

RNN(
  (embedding): Embedding(23449, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [39]:
def train(dataloader, optimizer, loss_fn):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += (
            (pred >= 0.5).float() == label_batch
        ).float().sum().item()
        total_loss += loss.item() * label_batch.size(0)
    return (
        total_acc / len(dataloader.dataset),
        total_loss / len(dataloader.dataset)
    )


def evaluate(dataloader, loss_fn):
    model.eval()
    total_acc, total_loss = 0, 0
    
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += (
                (pred >= 0.5).float() == label_batch
            ).float().sum().item()

    return (
        total_acc / len(dataloader.dataset),
        total_loss / len(dataloader.dataset)
    )


loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [40]:
num_epochs = 3
torch.manual_seed(1)

for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl, optimizer, loss_fn)
    acc_valid, loss_valid = evaluate(valid_dl, loss_fn)
    print(
        f'Epoch {epoch} accuracy: {acc_train:.4f}',
        f' val_accuracy: {acc_valid:.4f}'
    )

Epoch 0 accuracy: 0.8580  val_accuracy: 1.0000
Epoch 1 accuracy: 1.0000  val_accuracy: 1.0000
Epoch 2 accuracy: 1.0000  val_accuracy: 1.0000


In [ ]:
acc_test, _ = evaluate(test_dl, loss_fn)
print(f'test_accuracy: {acc_test:.4f}')